In [5]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from keras.utils import pad_sequences
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LAParams
import io
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm , trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from langchain.text_splitter import TokenTextSplitter
import nltk
# nltk.download('punkt')  # Download the punkt tokenizer for sentence tokenization
from nltk.tokenize import sent_tokenize
import math
from concurrent.futures import ThreadPoolExecutor
import language_tool_python
import requests


2023-07-27 12:37:40.690599: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-27 12:37:40.742825: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 12:37:41.552666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [25]:
def grammer_correction(text):
    tool = language_tool_python.LanguageTool('en-US')
    # text = 'I am going to Goa.'
    matches = tool.check(text)
    len(matches)
    tool.close() # Call `close()` to shut off the server when you're done.
    if len(matches):
        return 0
    return matches


In [6]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        resource_manager = PDFResourceManager()
        fake_file_handle = io.StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)
        
        for page in PDFPage.get_pages(file, check_extractable=True):
            page_interpreter.process_page(page)
            text += fake_file_handle.getvalue()
        
        converter.close()
        fake_file_handle.close()
    
    return text
# Function to divide text into chunks of 512 tokens
def divide_text_into_chunks(text, max_chunk_size=512):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        # print(sentence)
        # print("##########")
        tokenized_sentence = sentence.split()
        if len(current_chunk) + len(tokenized_sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def divide_text_into_batches(text, batch_size=5):
    sentences = nltk.sent_tokenize(text)
    num_sentences = len(sentences)
    num_batches = math.ceil(num_sentences / batch_size)

    batches = []
    start_idx = 0
    for batch_idx in range(num_batches):
        end_idx = min(start_idx + batch_size, num_sentences)
        batch_sentences = sentences[start_idx:end_idx]
        batches.append(" ".join(batch_sentences))
        start_idx = end_idx

    return batches


In [7]:
def check_text_local(input_text, language='en-US'):
    url = 'http://localhost:8081/v2/check'
    params = {'text': input_text, 'language': language}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data['matches']
    else:
        print("Error:", response.text)
        return []

In [8]:
pdf_path = './15031-4983-FullBook.pdf'
text = extract_text_from_pdf(pdf_path)
chunks = divide_text_into_chunks(text)
batches = divide_text_into_batches(text, 10)
print(len(chunks))
print(len(batches))
corrected_text = check_text_local(text)


76437
33193


In [26]:
grammer_matches = []
for idx, chunk in enumerate(batches):
    print("batch Number :",idx)
    sentences = sent_tokenize(chunk)
    # chunk_label = check_grammar(sentences, GED_model, GED_tokenizer)
    for i in sentences:
        text = grammer_correction(i)
        if not text:
            grammer_matches.append(i)
            grammer_matches.append(text)

batch Number : 0
batch Number : 1
batch Number : 2


KeyboardInterrupt: 

In [2]:
from languagetool import LanguageTool

def check_text_local(input_text, language='en-US'):
    tool = LanguageTool('http://localhost:8081', language)
    errors = tool.check(input_text)
    return errors

if __name__ == "__main__":
    text_to_check = "Learner Voice, Learner Choice offers fresh, forward-thinking"




ModuleNotFoundError: No module named 'languagetool'

In [3]:
import requests



if __name__ == "__main__":
    text_to_check = "Learner Voice, Learner Choice offers fresh, forward-thinking supports for teachers creating an empowered, student-centered classroom."
    errors = check_text_local(text_to_check)
    print("Number of errors:", len(errors))
    for error in errors:
        print(f"{error['rule']['id']}: {error['message']} (Line: {error['context']['offset']}, Column: {error['context']['offset']})")


Number of errors: 0
